## Analiza danych tekstowych - text mining

Pobierz z wikipedii treść 11 artykułów.

In [ ]:
!pip install requests

In [ ]:
# from urllib.request import urlopen
import requests


wiki_url = "http://en.wikipedia.org/wiki/"
titles = [
    "Integral", 
    "Riemann_integral", 
    "Riemann-Stieltjes_integral", 
    "Derivative",
    "Limit_of_a_sequence", 
    "Edvard_Munch", 
    "Vincent_van_Gogh", 
    "Jan_Matejko",
    "Lev_Tolstoj", 
    "Franz_Kafka", 
    "J._R._R._Tolkien"
]

urls = [wiki_url + title for title in titles]

# articles = [urlopen(url).read() for url in urls]
articles = [requests.get(url).content for url in urls ]

# Wyświetlamy pierwsze 200 znaków pierwszego artykułu
print(articles[0][:100])

Wczytaliśmy cały kod strony artykułu (włącznie z informacjami o nagłówkach, paskami nawigacyjnymi, ... ). Nas interesuje wyłącznie sama treść artykułów. W jaki sposób można wyciągnąć z kodu źródłowego strony wybrane informacje?

Proces mający na celu wyciągnięcie z nieustrukturyzowanego zbioru danych tekstowych (a kod źródłowy strony internetowej jest takim zbiorem) wybranych informacji nazywany jest **scrapingiem**. W Pythonie najpopularniejszą biblioteką do scrapingu jest BeautifulSoup.

Żeby "zeskrapować" jakiś tekst, najpierw trzeba rozpoznać w nim charakterystyczne elementy. Przeglądając kod źródłowy artykułów zauważymy, że artykuły na wiki są zamknięte w elemencie `<div>` o `id=bodyContent`, a poszczególne akapity artykułu to po prostu paragrafy (`<p>`). 

Czyli teraz z całej tej pobranej treści chcemy wyciągnąć wyłącznie elementy `<p>` znajdujące się wewnątrz elementu `<div>` o id `"bodyContent"`. Do tego właśnie służy biblioteka BeautifulSoup.

In [ ]:
from bs4 import BeautifulSoup

articles_paragraphs = [BeautifulSoup(article).find("div", id="bodyContent").find_all("p") for article in articles]

# wyświetlmy pierwszy paragraf pierwszego artykułu
print(articles_paragraphs[0][0])

Każdy paragraf jest reprezentowany jako obiekty klasy Tag biblioteki BeautifulSoup.

In [ ]:
print(type(articles_paragraphs[0][0]))

Zrzutujmy paragrafy na typ string.

In [ ]:
articles_paragraphs = [[str(paragraph) for paragraph in paragraphs] for paragraphs in articles_paragraphs]

Teraz każdy artykuł mamy porozbijany na zbiór paragrafów. Sklejmy te paragrafy tak, żeby artykuły znów stały się jednym ciągiem znaków.

In [ ]:
scraped_articles = ["".join(paragraphs) for paragraphs in articles_paragraphs]

# wyświetlmy pierwsze 200 znaków pierwszego, tak przetworzonego artykułu
print(scraped_articles[0][:200])

Pozbądźmy się z tekstu znaczników html, tak żeby w paragrafach został już czysty tekst. Do tego celu użyjemy biblioteki `re` (pythonowej biblioteki do pracy z wyrażeniami regularnymi).

Za wzorzec znacznika html przyjmujemy `<.+?>` (znak "<" po którym następuje jedne lub więcej znaków, kończący się znakiem ">" - leniwe)

In [ ]:
import re

cleaned_articles = [re.sub("<.+?>", "", article) for article in scraped_articles]

# wyświetlmy piersze 5000 znaków pierwszego, oczyszczonego w ten sposób artykuł
print(cleaned_articles[0][:5000])

Jeszcze trochę preporcessingu.

Zamieńmy wielkie litery na małe

In [ ]:
cleaned_articles = [a.lower() for a in cleaned_articles]

# wyświetlmy piersze 200 znaków pierwszego artykułu
print(cleaned_articles[0][:200])

Tokenizacja

In [ ]:
from nltk.tokenize import word_tokenize

cleaned_articles = [word_tokenize(article) for article in cleaned_articles]

# wyświetlmy pierwsz 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

Usuńmy znaki interpunkcyjne

In [ ]:
import string

cleaned_articles = [[token for token in article if token not in string.punctuation] for article in cleaned_articles]

# wyświetlmy piersze 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

Usuwamy stopwords

In [ ]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')

cleaned_articles = [[token for token in article if token not in stopwords_list] for article in cleaned_articles]

# wyświetlmy pierwsze 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

Stemming

In [ ]:
from nltk.stem import PorterStemmer  # najpopularniejszy stemmer

stemmer = PorterStemmer()
cleaned_articles = [[stemmer.stem(token) for token in article] for article in cleaned_articles]

# wyświetlmy pierwsze 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

#### Embedding (osadzanie/wektoryzacja)

tf-idf

In [ ]:
!pip install gensim

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

dictionary = Dictionary(cleaned_articles)
bow_corpus = [dictionary.doc2bow(article) for article in cleaned_articles]
tfidf_model = TfidfModel(bow_corpus)

# Wyświetlmy pierwsze 100 tokenów pierwszego artykułu
tfidf_corpus = tfidf_model[bow_corpus[0]][:100]
print(tfidf_corpus)

Wyświetlmy pierwsze 10 najczęściej występujących tokenów z pierwszego artykułu

In [ ]:
first_article = sorted(tfidf_model[bow_corpus[0]], key=lambda x: x[1], reverse=True)[:10]
first_article

Co to za słowa ?

In [ ]:
[dictionary[token[0]] for token in first_article]

In [ ]:
tfidf_corpus = tfidf_model[bow_corpus]

### Analiza semantyczna

Jednym z kolejnych etapów może być analiza semantyczna (czyli znaczeniowa) przetwarzanych treści. Częstym zadaniem, które w tym miejscu pojawiaja się jest dopasowywanie tytułów (kategorii) do treści. Jednym z algorytmów odnoszących świetne wyniki w tej dziedzinie jest algorytm LDA (*ang. Latent Dirichlet allocation*), który do znalezienia najlepiej pasujących tytułów wykorzystuje analizę PCA podanego mu rozkładu tfidf lub bow. Biblioteka Gensim posiada implementację tego algorytmu. Wystarczy przekazać do niej słownik oraz wynik zwrócony przez model tfidf lub bow.

In [ ]:
from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(
    corpus=tfidf_corpus,
    id2word=dictionary,
    num_topics=30,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto"
)
lda_model

W modelu lda przyjęliśmy wartość parametru `num_topics` 30, co oznacza, że model będzie szukał 30 najlepszych tytułów dla całego korpusu. Tytuł będzie zbudowanych z najczęściej występujących w tokenów. W pythonie istnieje biblioteka (pyldavis) do wizualizacji wyników zwróconych przez model. Wyświetlmy wynik na wykresie.

In [ ]:
pip install pyldavis

In [ ]:
#vis
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(
    lda_model, 
    tfidf_corpus, 
    dictionary, 
    mds="mmds", 
    R=30  # liczba tokenów branych pod uwagę podczas analizy tytułu dla wybranego artykułu
)
vis

Każdy ze znajdujących się po lewej stronie okręgów reprezentuje jakiś potencjalny tytuł (zbiór tokenów, z których ten tytuł można zbudować). Chcieliśmy znaleźć 30 propozycji tytułów. Algorytmowi udało się znaleźć 8. Niektóre z nich (np. 3 i 4) nakładają się. Im bardziej tytuły będą od siebie odseparowane, tym lepiej. Po kliknięciu w wybraną propozycję (okrąg) po prawej stronie zobaczymy słowa powiązane z tym tytułem i ich wagi.

Wynik mógłby być lepszy. Pierwszą rzeczą, którą napewno warto zrobić pod kątem poprawienia wyniku jest usunięcie tokenów, które nie niosą ze sobą wiele znaczenia, a nie było ich na liście stopwords. W przypadku propozycji tytułu numer 1 będą to napewno tokeny takie jak 'f', 'x', 'g', 'b', '`', ... Warto też pewnie zmniejszyć liczbę słów, z których próbujemy zbudować tytuł. Zostawmy to ćwiczenie do samodzielnego rozwiązania, bo jest ono dobrym sposobem na utrwalenie materiału.
